Intro 

Libriaries and Imports 

In [4]:
!pip install beautifulsoup4
!pip install lxml
import requests 
import pandas as pd
import numpy as np 
import random 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


Solving environment: done

# All requested packages already installed.

Solving environment: | 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



\ 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Scrapping

In [12]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
tab = str(soup.table)


In [6]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Cleaning

In [7]:
df1 = df[df.Borough != 'Not assigned']

# Combining
df2 = df1.groupby(['Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)


df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2

,Borough,Postal Code,Neighbourhood
0,North York,"M3A, M4A, M6A, M3B, M6B, M3C, M2H, M3H, M2J, M...","Parkwoods, Victoria Village, Lawrence Manor, L..."
1,Downtown Toronto,"M5A, M7A, M5B, M5C, M5E, M5G, M6G, M5H, M5J, M...","Regent Park, Harbourfront, Queen's Park, Ontar..."
2,Etobicoke,"M9A, M9B, M9C, M9P, M9R, M8V, M9V, M8W, M9W, M...","Islington Avenue, Humber Valley Village, West ..."
3,Scarborough,"M1B, M1C, M1E, M1G, M1H, M1J, M1K, M1L, M1M, M...","Malvern, Rouge, Rouge Hill, Port Union, Highla..."
4,East York,"M4B, M4C, M4G, M4H, M4J","Parkview Hill, Woodbine Gardens, Woodbine Heig..."
5,York,"M6C, M6E, M6M, M6N, M9N","Humewood-Cedarvale, Caledonia-Fairbanks, Del R..."
6,East Toronto,"M4E, M4K, M4L, M4M, M7Y","The Beaches, The Danforth West, Riverdale, Ind..."
7,West Toronto,"M6H, M6J, M6K, M6P, M6R, M6S","Dufferin, Dovercourt Village, Little Portugal,..."
8,Central Toronto,"M4N, M5N, M4P, M5P, M4R, M5R, M4S, M4T, M4V","Lawrence Park, Roselawn, Davisville North, For..."
9,Mississauga,M7R,Canada Post Gateway Processing Centre


Shape

In [8]:
df2.shape

(10, 3)

Lat & Log

In [9]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')

lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df3 = df2[df2['Borough'].str.contains('Toronto',regex=False)]
df3

,Borough,Postal Code,Neighbourhood
1,Downtown Toronto,"M5A, M7A, M5B, M5C, M5E, M5G, M6G, M5H, M5J, M...","Regent Park, Harbourfront, Queen's Park, Ontar..."
6,East Toronto,"M4E, M4K, M4L, M4M, M7Y","The Beaches, The Danforth West, Riverdale, Ind..."
7,West Toronto,"M6H, M6J, M6K, M6P, M6R, M6S","Dufferin, Dovercourt Village, Little Portugal,..."
8,Central Toronto,"M4N, M5N, M4P, M5P, M4R, M5R, M4S, M4T, M4V","Lawrence Park, Roselawn, Davisville North, For..."


In [11]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng in zip(lat_lon['Latitude'],lat_lon['Longitude']):
    label = '{}, {}'.format(lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

Clustering

In [31]:
k=5

toronto_clustering = lat_lon.drop(['Postcode'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
lat_lon.insert(0, 'Cluster Labels', kmeans.labels_)

Mapping

In [39]:

map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)


x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, cluster in zip(lat_lon['Latitude'],lat_lon['Longitude'], lat_lon['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters